In [167]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itr
from boundaries import WallBoundary

# Initializing a rank three array filled with nodes
latticeSize = 9
xResolution = 5
yResolution = 5
# Weights
cxs = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1])
cys = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1])
unitVeloVect = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [-1, 0],
    [0, -1],
    [1, 1],
    [-1, 1],
    [-1, -1],
    [1, -1]
])
cs = np.sqrt(3)
weight = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36])

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 13)

In [200]:
# One iteration only

iteratedFluid = np.ones((xResolution, yResolution, latticeSize))
# Internal collision step
density = np.sum(fluid, axis = 2)
mesoMomentum = fluid.reshape(xResolution, yResolution, latticeSize, 1) * unitVeloVect
momentum = np.sum(mesoMomentum, axis = 2) / density.reshape(xResolution, yResolution, 1)
momentumMagnitude = np.linalg.norm(momentum, axis = 2) / (2 * cs**4)

In [201]:
a = np.array([
	[[1, 1], [2, 2]],
	[[3, 3], [4, 4]]
])
b = np.array([
	[[1, 1], [2, 2]],
	[[3, 3], [4, 4]]
])
adotb = np.einsum("ijk, ijk -> ij", a, b)

TypeError: Field elements must be 2- or 3-tuples, got '[3, 3]'